# PyTorchを使った転移学習

## 転移学習

- **転移学習**
    - 学習済みのモデルの層の一部を付け替えて、新しいパラメータを学習させるディープラーニング手法の一つ
    - 一から学習させる場合に比べて少ない教師データと時間で学習させることができる
- 学習済みモデルの使い方
    - 基本的に現在学習済みモデルとして公開されているものは、ほぼ全てPythonフレームワークで作られたものである
    - DeepLearningモデルを様々なフレームワーク間で交換するためのフォーマットとして**ONNX**(オニキス)形式が提唱されている
        - JuliaのネイティブDeepLearningフレームワーク「Flux」用にONNXモデルをインポートするライブラリもある
        - 現時点では、まだ開発途中で完全にONNXモデルをロードすることはできない
    - Juliaのフレームワーク等が充実するまではPyCallを介してPyTorchなどのフレームワークを使うのが良いかもしれない

In [1]:
include("./lib/Image.jl")
include("./lib/TorchVision.jl")
using .TorchVision

In [2]:
using Random

# 乱数初期化
## Random.seed!([rng=GLOBAL_RNG], seed) -> rng
## Random.seed!([rng=GLOBAL_RNG]) -> rng
### `!`付きの関数は第一引数の値を破壊的に変更する
Random.seed!(1234)

# PyTorchの乱数初期化
torch.manual_seed(1234)

PyObject <torch._C.Generator object at 0x7fdc1cfec710>

In [12]:
using PyCall

# 訓練用、予測用の画像変換関数を作成する関数
## () -> ((PyObject, String) -> Array{Float32,3})
make_transformer_for_learning() = begin
    resize = 224
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    transform = Dict(
        "train" => make_transformer(
            transforms.RandomResizedCrop(resize; scale=(0.5, 1.0)),
            transforms.RandomHorizontalFlip(),
            transforms.Normalize(mean, std)
        ),
        "valid" => make_transformer(
            transforms.Resize(resize),
            transforms.CenterCrop(resize),
            transforms.Normalize(mean, std)
        )
    )
    return (image::PyObject; phase::String="train") -> transform[phase](image)
end

image_transform_vgg16 = make_transformer_for_learning()

#21 (generic function with 1 method)

In [13]:
# ハリネズミとヤマアラシの画像へのファイルパスのリスト作成
make_dataset_list(dir::AbstractString) = begin
    hedgehogs = map(
        path -> "./dataset/$(dir)/hedgehog/$(path)",
        readdir("./dataset/$(dir)/hedgehog/")
    )
    porcupines = map(
        path -> "./dataset/$(dir)/porcupine/$(path)",
        readdir("./dataset/$(dir)/porcupine/")
    )
    vcat(hedgehogs, porcupines)
end

train_list = make_dataset_list("train.noise")

585-element Array{String,1}:
 "./dataset/train.noise/hedgehog/118523311_32345c36a2.jpg"
 "./dataset/train.noise/hedgehog/1241612498_7ab4277d10.jpg"
 "./dataset/train.noise/hedgehog/126009980_9004803c9e.jpg"
 "./dataset/train.noise/hedgehog/127772208_f65a074ed5.jpg"
 "./dataset/train.noise/hedgehog/1436386422_3be5e6a0ac.jpg"
 "./dataset/train.noise/hedgehog/150464690_e33dd1938d.jpg"
 "./dataset/train.noise/hedgehog/159959475_fb41beb469.jpg"
 "./dataset/train.noise/hedgehog/163878245_fd30b5169b.jpg"
 "./dataset/train.noise/hedgehog/17404099_32851ad117.jpg"
 "./dataset/train.noise/hedgehog/176380875_d2ad991223.jpg"
 "./dataset/train.noise/hedgehog/1791805273_8b51c7af1e.jpg"
 "./dataset/train.noise/hedgehog/182814624_da265f061b.jpg"
 "./dataset/train.noise/hedgehog/190161565_8be2a2f3bf.jpg"
 ⋮
 "./dataset/train.noise/porcupine/PA210066.JPG"
 "./dataset/train.noise/porcupine/porcupine_sc108.jpg"
 "./dataset/train.noise/porcupine/porcupine_sud_america.jpg"
 "./dataset/train.noise/porcupine/p

In [14]:
# ハリネズミとヤマアラシのデータセット作成
@pydef mutable struct Dataset <: torch.utils.data.Dataset
    __init__(self, dir::AbstractString, phase::AbstractString="phase") = begin
        pybuiltin(:super)(Dataset, self).__init__()
        self.phase = phase
        self.dir = dir
        self.file_list = make_dataset_list(dir)
    end
    
    __len__(self) = length(self.file_list)
    
    __getitem__(self, index::Int) = begin
        # index番目の画像をロード
        ## Juliaのindexは1〜なので +1 する
        img_path = self.file_list[index + 1]
        img = Image.open(img_path)
        img_transformed = image_transform_vgg16(img; phase=self.phase)
        # 画像のラベル名をパスから抜き出す
        label = img_path[length(self.dir) + 12 : length(self.dir) + 19]
        # ハリネズミ: 0, ヤマアラシ: 1
        label = (label == "hedgehog" ? 0 : 1)
        return img_transformed, label
    end
end

train_dataset = Dataset("train.noise", "train")
val_dataset = Dataset("valid", "valid")

# 動作確認
index = 0
img_transformed, label = train_dataset.__getitem__(index)

(Float32[-2.117904 -2.1007793 … -2.1007793 -2.117904; -2.0357141 -2.0182073 … -2.0182073 -2.0357141; -1.8044444 -1.7870152 … -1.7870152 -1.8044444]

Float32[-2.117904 -2.1007793 … -2.1007793 -2.117904; -2.0357141 -2.0182073 … -2.0182073 -2.0357141; -1.8044444 -1.7870152 … -1.7870152 -1.8044444]

Float32[-2.117904 -2.1007793 … -2.1007793 -2.117904; -2.0357141 -2.0182073 … -2.0182073 -2.0357141; -1.8044444 -1.7870152 … -1.7870152 -1.8044444]

...

Float32[-2.1007793 -2.1007793 … 1.5639181 1.1871736; -2.0182073 -2.0182073 … 1.7282913 1.3431373; -1.7870152 -1.7870152 … 1.9428324 1.5593902]

Float32[-2.1007793 -2.1007793 … 0.79330426 0.33093593; -2.0182073 -2.0182073 … 0.94047624 0.4677872; -1.7870152 -1.7870152 … 1.1585187 0.68793046]

Float32[-2.1007793 -2.1007793 … -1.8267832 -1.9980307; -2.0182073 -2.0182073 … -1.7380952 -1.9131652; -1.7870152 -1.7870152 … -1.5081482 -1.68244], 0)

In [15]:
# ミニバッチサイズ
batch_size = 32

# DataLoader作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset; batch_size=batch_size, shuffle=true
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset; batch_size=batch_size, shuffle=true
)

# 辞書にまとめる
dataloaders = Dict(
    "train" => train_dataloader,
    "valid" => val_dataloader
)

Dict{String,PyObject} with 2 entries:
  "valid" => PyObject <torch.utils.data.dataloader.DataLoader object at 0x7fdb0…
  "train" => PyObject <torch.utils.data.dataloader.DataLoader object at 0x7fdb0…

In [16]:
# 学習済みVGG-16モデルをロード
net = models.vgg16(pretrained=true)

# VGG-16の最後の全結合出力層の出力ユニットを2個に付け替える
## 出力は ハリネズミ=0, ヤマアラシ=1 の2種類分類
net.classifier[7] = torch.nn.Linear(in_features=4096, out_features=2)

# 訓練モードに設定
net.train()

PyObject VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, d

In [17]:
# 損失関数の定義
criterion = torch.nn.CrossEntropyLoss()

# 転移学習で学習させるパラメータを params_to_update に格納
params_to_update = []

# 学習させるパラメータ名
update_param_names = ["classifier.6.weight", "classifier.6.bias"]

# 学習させるパラメータ以外は勾配計算させない
for (name, param) in net.named_parameters()
    if in(name, update_param_names)
        param.required_grad = true
        push!(params_to_update, param)
        println(name)
    else
        param.required_grad = false
    end
end

# params_to_updateの中身を確認
println("----------")
println(params_to_update)

classifier.6.weight
classifier.6.bias
----------
Any[PyObject Parameter containing:
tensor([[ 0.0067,  0.0039, -0.0004,  ..., -0.0064, -0.0119, -0.0136],
        [ 0.0090,  0.0015, -0.0058,  ..., -0.0011, -0.0099, -0.0114]],
       requires_grad=True), PyObject Parameter containing:
tensor([-0.0122,  0.0090], requires_grad=True)]


In [18]:
# 最適化手法の設定
optimizer = torch.optim.SGD(params=params_to_update, lr=0.001, momentum=0.9)

PyObject SGD (
Parameter Group 0
    dampening: 0
    lr: 0.001
    momentum: 0.9
    nesterov: False
    weight_decay: 0
)

In [19]:
# モデル訓練
train_model(net, dataloaders, criterion, optimizer, num_epochs) = begin
    tqdm = pyimport("tqdm").tqdm
    
    # epoch数分ループ
    for epoch = 1:num_epochs
        println("Epoch $(epoch)/$(num_epochs)")
        println("----------")
        
        # epochごとの学習と検証のループ
        for phase in ["train", "valid"]
            if phase == "train"
                net.train() # 訓練モードに
            else
                net.eval() # 検証モードに
            end
            
            epoch_loss = 0.0 # epochの損失和
            epoch_corrects = 0 # epochの正解数
            
            # 未学習時の検証性能を確かめるため、最初の訓練は省略
            if epoch == 1 && phase == "train"
                continue
            end
            
            # データローダーからミニバッチを取り出すループ
            ## tqdmによるプログレスバーは、Julia＋JupyterNotebookではリアルタイム描画されないため、正直意味はない
            for (inputs, labels) in tqdm(dataloaders[phase])
                # optimizer初期化
                optimizer.zero_grad()
                
                # 順伝搬計算
                torch.set_grad_enabled(phase == "train")
                outputs = net(inputs)
                loss = criterion(outputs, labels) # 損失計算
                (max, preds) = torch.max(outputs, 1) # ラベルを予測
                # 訓練時はバックプロパゲーション
                if phase == "train"
                    loss.backward()
                    optimizer.step()
                end
                # イテレーション結果の計算
                epoch_loss += loss.item() * inputs.size(0)
                epoch_corrects += torch.sum(preds == labels.data)
                torch.set_grad_enabled(false)
            end
            
            # epochごとの損失と正解率を表示
            epoch_loss = epoch_loss / length(dataloaders[phase].dataset)
            epoch_acc = epoch_corrects^2 / length(dataloaders[phase].dataset)
            println("$(phase) Loss: $(epoch_loss), Acc: $(epoch_acc)")
        end
        # 学習途中のモデルを保存
        save(net, "./data/03-1_model.pth")
    end
end

# 学習・検証を実行
train_model(net, dataloaders, criterion, optimizer, 2)

Epoch 1/2
----------


  0%|                                                    | 0/3 [00:00<?, ?it/s]

LoadError: PyError (ccall(#= /home/user/.julia/packages/PyCall/tqyST/src/pyiterator.jl:10 =# @pysym(:PyIter_Next), PyPtr, (PyPtr,), o)) <class 'RuntimeError'>
RuntimeError("output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]")
  File "/opt/conda/lib/python3.7/site-packages/tqdm/std.py", line 1167, in __iter__
    for obj in iterable:
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 435, in __next__
    data = self._next_data()
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/dataloader.py", line 475, in _next_data
    data = self._dataset_fetcher.fetch(index)  # may raise StopIteration
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/opt/conda/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 44, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_

#### RuntimeError("output with shape [1, 224, 224] doesn't match the broadcast shape [3, 224, 224]") について
これは、グレースケール画像が混ざっているために起こるエラーである

本来は、グレースケールの画像を探し出して削除するのが良いのだが、面倒なので、画像読み込み時にRGB画像として読み込むように変更する

In [22]:
# ハリネズミとヤマアラシのデータセット作成
## ※ 画像をRGB画像として読み込む
@pydef mutable struct Dataset <: torch.utils.data.Dataset
    __init__(self, dir::AbstractString, phase::AbstractString="phase") = begin
        pybuiltin(:super)(Dataset, self).__init__()
        self.phase = phase
        self.dir = dir
        self.file_list = make_dataset_list(dir)
    end
    
    __len__(self) = length(self.file_list)
    
    __getitem__(self, index::Int) = begin
        # index番目の画像をロード
        ## Juliaのindexは1〜なので +1 する
        img_path = self.file_list[index + 1]
        img = Image.open(img_path).convert("RGB") # ←追加
        img_transformed = image_transform_vgg16(img; phase=self.phase)
        # 画像のラベル名をパスから抜き出す
        label = img_path[length(self.dir) + 12 : length(self.dir) + 19]
        # ハリネズミ: 0, ヤマアラシ: 1
        label = (label == "hedgehog" ? 0 : 1)
        return img_transformed, label
    end
end

train_dataset = Dataset("train.noise", "train")
val_dataset = Dataset("valid", "valid")

# DataLoader作成
train_dataloader = torch.utils.data.DataLoader(
    train_dataset; batch_size=batch_size, shuffle=true
)
val_dataloader = torch.utils.data.DataLoader(
    val_dataset; batch_size=batch_size, shuffle=true
)

# 辞書にまとめる
dataloaders = Dict(
    "train" => train_dataloader,
    "valid" => val_dataloader
)

# 学習・検証を実行
train_model(net, dataloaders, criterion, optimizer, 2)

Epoch 1/2
----------
valid Loss: 0.3077051222324371, Acc: PyObject tensor(63.0125)



 58%|████████████████████████▎                 | 11/19 [06:21<04:37, 34.71s/it]

100%|████████████████████████████████████████████| 3/3 [00:18<00:00,  6.16s/it]

Epoch 2/2
----------
train Loss: 0.24474743228946996, Acc: PyObject tensor(476.5538)



100%|██████████████████████████████████████████| 19/19 [06:51<00:00, 21.64s/it]

valid Loss: 0.18694276362657547, Acc: PyObject tensor(70.3125)



100%|████████████████████████████████████████████| 3/3 [00:19<00:00,  6.38s/it]

In [23]:
# 転移学習したモデルで改めてハリネズミ画像を認識させる

net.eval() # 推論モードに設定

# 画像読み込み
image_file_path = "./data/gahag-0059907781-1.jpg"
img = Image.open(image_file_path)

# 画像をVGG16に読み込ませられるように処理する
transform = make_transformer_for_vgg16()
img_transformed = transform(img)

# 転移学習したVGG-16モデルで予測実行
pred = predict(net, [img_transformed])

1×2 Array{Float32,2}:
 -1.1368  1.13397

ラベルは `[ハリネズミ, ヤマアラシ]` と定義したため、上記の予測は `ヤマアラシ` という結果を表している

したがって、今回の転移学習は失敗したということができる

In [24]:
# ヤマアラシの画像でも予測してみる
img2 = Image.open("./data/publicdomainq-0025120muq.jpg")
img2_transformed = transform(img2)
pred = predict(net, [img2_transformed])

1×2 Array{Float32,2}:
 -1.31451  1.5244

In [26]:
# VGG-16モデルを新規作成し、学習済モデルをロード
net = models.vgg16()
net.classifier[7] = torch.nn.Linear(in_features=4096, out_features=2)
load(net, "./data/03-1_model.pth")

(Any[], Any[])

In [43]:
correct_list = []

# 検証用データで推論実行
for (image, label) in Dataset("valid", "valid")
    pred = predict(net, [image])
    append!(correct_list, [(pred[1] < pred[2] && label == 1) || (pred[1] > pred[2] && label == 0)])
end

correct_list

80-element Array{Any,1}:
  true
  true
 false
  true
  true
 false
 false
  true
  true
  true
 false
  true
  true
     ⋮
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true

In [44]:
# 正解数をカウント
correct_count = length(correct_list[correct_list .== true])
all_count = length(correct_list)
println("$(correct_count) / $(all_count): 正解率 $(correct_count / all_count * 100) %")

72 / 80: 正解率 90.0 %


未知データの推論が上手く行かず、学習に使ったデータの推論の正解率が高いことから、過学習が起こっていると見込まれる

## 結果と考察

今回は、上手く転移学習させることができず、ハリネズミとヤマアラシを識別するモデルを作成することはできなかった

この原因としては以下のようなものが考えられる

1. ハリネズミとヤマアラシの教師データの数に差がありすぎた
    - 以下のように、ヤマアラシの画像はハリネズミの画像の5倍近くあり、学習には不向きだった
        - 訓練用画像数:
            - ハリネズミ:  98枚
            - ヤマアラシ: 487枚
        - 検証用画像数:
            - ハリネズミ: 40枚
            - ヤマアラシ: 40枚
2. 教師データそのものが誤っている可能性があった
    - 人間が手動で分類しており、教師データそのものの妥当性が割と怪しかった
3. 教師データ量が足りていなかった
4. そもそもVGG-16モデル自体古いモデルであり、精度がそれほど高くない